In [6]:
import os

import numpy as np
import tensorflow as tf
import trimesh
import tensorboard
from feature_detect.src.feat_data_keras import Data_Gen

from tensorboard.plugins.mesh import summary as mesh_summary
tf.compat.v1.disable_eager_execution()

npz_path='F:/ProjectData/mesh_feature/test/test_npz/front'
log_dir = 'F:/ProjectData/mesh_feature/test/log_dir'
batch_size = 1



In [7]:
# Camera and scene configuration.
config_dict = {
    'material': {
      'cls': 'PointsMaterial',
      'size': 1,
        # 'color':0x000fff
    }
}


# Read all sample PLY files.

In [8]:
data_gen = Data_Gen(npz_path)
data,  npz_name,epoch_end = data_gen.load_pkg()

# vertices=data['x'].astype(np.float32)
# features=data['y'].astype(np.float32)
# 
# # Add batch dimension, so our data will be of shape BxNxC.
# points_v = vertices
# points_f=features[:,1:]

vertices=data['x'][1].astype(np.float32)
features=data['y'][1].astype(np.float32)

# Add batch dimension, so our data will be of shape BxNxC.
points_v = np.expand_dims(vertices, 0)
points_f=np.expand_dims(features[:,1:],0)
points=np.concatenate([points_v,points_f],axis=1)


colors_v = np.ones_like(points_v)*[0,0,255]
colors_f = np.ones_like(points_f)*[0,255,0]
colors=np.concatenate([colors_v,colors_f],axis=1)


In [9]:
points_tensor = tf.compat.v1.placeholder(tf.float32, points.shape)
colors_tensor = tf.compat.v1.placeholder(tf.int32, colors.shape)

summary = mesh_summary.op(
    'v_color_tensor', 
    vertices=points_tensor,
    colors=colors_tensor, 
    config_dict=config_dict
)

# Create summary writer and session.
writer = tf.compat.v1.summary.FileWriter(log_dir)
sess = tf.compat.v1.Session()




In [10]:
summaries = sess.run([summary], feed_dict={
    points_tensor: points,
    colors_tensor: colors,
})
# Save summaries.
for summary in summaries:
  writer.add_summary(summary)

